In [1]:
import sys
import os

# Adding src path so we can import modules
src_path = os.path.abspath(os.path.join('..', '..',))

if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
from core.base_urls import SEARCHING_URL
from core.datasets import dataset_1_test
from utils.extracting import extract_doc_ids
from measures.precision import get_precision_at_k
from measures.recall import get_recall, get_avg_recall
from measures.map import get_ap_at_k, get_map
from measures.mrr import get_reciprocal_rank, get_mrr
import requests

queries = dataset_1_test.queries_iter()
qrels = dataset_1_test.qrels_iter()

count = 0
recalls = []
aps = []
rrs = []

for query in queries:
    if count == 10:
        break

    query_id = query.query_id
    print(f'processing {query_id}')

    # get matching results
    search_text = query.text
    request_body = {
        'dataset_id': 1,
        'search_text': search_text,
    }
    response = requests.post(SEARCHING_URL, json = request_body)    

    if response.status_code == 200:
        # get retrieved docs
        retrieved_docs = response.json()['data']
        retrieved_ids = extract_doc_ids(retrieved_docs)

        print('Retrieved IDs')
        print(retrieved_ids)

        # get relevant docs
        # NO CONDITION ON REVELANCE BECAUSE DATASET HAS ONLY QRELS OF RELEVANCE = 1
        relevant_ids = [qrel.doc_id for qrel in qrels if qrel.query_id == query_id]
        # convert ids to int
        relevant_ids = [int(id) for id in relevant_ids]

        print('Relevant IDs')
        print(relevant_ids)

        # calc precision@10
        precision = get_precision_at_k(retrieved_ids, relevant_ids, 10)
        print(f'Precision@10 is: {precision}')

        # calc recall
        recall = get_recall(retrieved_ids, relevant_ids)
        print(f'Recall is: {recall}')
        recalls.append(recall)

        # calc ap@10
        ap = get_ap_at_k(retrieved_ids, relevant_ids, 10)
        print(f'AP@10 is: {ap}')
        aps.append(ap)

        # calc rr
        rr = get_reciprocal_rank(retrieved_ids, relevant_ids)
        print(f'RR is: {rr}')
        rrs.append(rr)
        print('-------------------------------------')

        count = count + 1

# calc for all queries' recalls
avg_recall = get_avg_recall(recalls)
print(f'Average Recall is: {avg_recall}')

# calc map
map = get_map(aps)
print(f'MAP@10 is: {map}')

# calc mrr
mrr = get_mrr(rrs)
print(f'MRR is: {mrr}')

processing 152
Retrieved IDs
[43849, 468356, 1117164, 1375828, 1940119, 2104807, 3188479, 3602237, 3782101, 4853061, 5072804, 6014242, 6443785, 7021576, 7362741, 8660233, 9627851, 9704560, 10092382, 10271274, 13391547, 13398821, 14821387, 16271350, 16357786, 17190300, 20148252, 20649332, 21148128, 21292979, 21577731, 22368728, 22394027, 22622597, 22798538, 23304312, 23684700, 23926423, 24292756, 27229377, 27397030, 27909271, 28344827, 29372030, 30063538, 30863530, 31778094, 33352043, 33532464, 33541817, 33792752, 33948002, 34439954, 36546686, 37164550, 37729380, 38778468, 39483099, 39540459, 40074690, 41041709, 41076383, 41677435, 41701432, 43667097, 43692108, 44420919, 44750421, 44797423, 45448520, 47143775, 47653868, 48422778, 48432306, 49098297, 49337071, 50914470, 54895186, 56179024, 56369247, 56542832, 56585748, 56939117, 57499409, 57839238, 57864964, 58124647, 58975000, 59918383, 60006359, 60623569, 61095254, 61657475, 63645279, 66412215, 66947886, 67949246, 68220312, 68717580, 6